# Day 17 - Formula 1 API

In [25]:
import requests
import json
import pandas as pd
import altair as alt

url_weather = "https://api.openf1.org/v1/weather"
url_meetings = "https://api.openf1.org/v1/meetings"



params_meetings = {
    "year": 2025,
    # Choose country name
    "country_name": "Austria"
}


response_meetings = requests.get(url_meetings, params=params_meetings)

# Check if correct status code retrieved
if response_meetings.status_code == 401:
    print("Formula 1 session in progress, no data available until session end")
elif response_meetings.status_code == 200:
    json_meetings = json.loads(response_meetings.text)
    
    df_meetings = pd.DataFrame(json_meetings)
    df_meetings
    
    # DataFrame of sessions for the Austrian Grand Prix
    meeting_id = df_meetings["meeting_key"]
    meeting_id

In [27]:
# Insert retieved meeting ID as weather api only takes meeting id or session id
params_weather = {
    "meeting_key": meeting_id
}

response_weather = requests.get(url_weather, params=params_weather)

json_weather = json.loads(response_weather.text)
    
df_weather = pd.DataFrame(json_weather)
df_weather

,date,session_key,wind_direction,track_temperature,air_temperature,pressure,wind_speed,meeting_key,rainfall,humidity
0,2025-06-27T11:16:09.524000+00:00,9948,62,37.2,23.2,944.2,1.7,1264,0,55.0
1,2025-06-27T11:17:09.536000+00:00,9948,48,37.1,23.1,944.2,1.4,1264,0,55.0
2,2025-06-27T11:18:09.530000+00:00,9948,24,36.9,23.3,944.1,2.3,1264,0,55.0
3,2025-06-27T11:19:09.537000+00:00,9948,53,36.8,23.3,944.1,2.6,1264,0,55.0
4,2025-06-27T11:20:09.550000+00:00,9948,70,37.0,23.4,944.2,1.3,1264,0,56.0
...,...,...,...,...,...,...,...,...,...,...
233,2025-06-28T11:31:15.366000+00:00,9950,0,40.8,25.6,946.5,0.4,1264,0,44.0
234,2025-06-28T11:32:15.386000+00:00,9950,80,41.3,25.8,946.4,0.7,1264,0,45.0
235,2025-06-28T11:33:15.362000+00:00,9950,0,42.2,26.0,946.4,1.6,1264,0,43.0
236,2025-06-28T11:34:15.364000+00:00,9950,293,42.0,26.0,946.4,1.1,1264,0,43.0


In [29]:
# Shows evolution of track temperature for the Austrian Grand Prix, First Practice 2.
track_temp = alt.Chart(df_weather).mark_line().encode(
    alt.X("date").title("Date"),
    alt.Y("track_temperature").title("Track temp")
)

air_temp = alt.Chart(df_weather).mark_line(color="black").encode(
    alt.X("date").title("Date"),
    alt.Y("air_temperature").title("Air temp")    
)

track_temp + air_temp

alt.LayerChart(...)

In [41]:
df_weather.dtypes

date                  object
session_key            int64
wind_direction         int64
track_temperature    float64
air_temperature      float64
pressure             float64
wind_speed           float64
meeting_key            int64
rainfall               int64
humidity             float64
dtype: object

In [42]:
df_weather["date"] = pd.to_datetime(df_weather["date"])

In [43]:
df_weather.dtypes

date                 datetime64[ns, UTC]
session_key                        int64
wind_direction                     int64
track_temperature                float64
air_temperature                  float64
pressure                         float64
wind_speed                       float64
meeting_key                        int64
rainfall                           int64
humidity                         float64
dtype: object

In [51]:
# Groupby by
df_weather.groupby(df_weather["date"].dt.day).agg({"track_temperature": ["min", "median", "max"]})

track_temperature             
                   min median   max
date                               
27                33.8   37.3  43.4
28                32.3   36.8  42.2